In [1]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import pandas as pd
import pickle
import torch.nn as nn
import torch
import time
from sklearn.metrics import precision_score, recall_score,f1_score
epochs = 17
batch_size = 2
output_dir = r'E:\2D_A_Result'
patience =5

#####TO GET IMAGE DATA
from skimage.external.tifffile import imread
from tqdm import tqdm
def get_data(folder):
    X = []
    y = []
    z = []
    for cell_type in os.listdir(folder):
        if not cell_type.startswith('.'):
            if cell_type in ['E']:
                label = 1
                label2 = 1
            elif cell_type in ['L']:
                label = 2
                label2 = 2
            elif cell_type in ['M']:
                label = 3
                label2 = 3
            elif cell_type in ['N']:
                label = 4
                label2 = 4
            elif cell_type in ['B']:
                label = 0
                label2 = 0
            for pro_name in tqdm(os.listdir(folder +'/'+ cell_type)):
                img_data= imread(folder + '/'+cell_type + '/' + pro_name)
                if img_data is not None:
                    X.append(img_data)
                    y.append(label)
                    z.append(label2)


#     X = np.asarray(X)
    X = np.asarray(X)/255
    y = np.asarray(y)
    z = np.asarray(z)
    return X,y,z

###To get image
data_path = r'F:\image\IMAGES'
x_img_16, y_img_16, z_img_16 = get_data(data_path)
train_index_path = r'C:\Users\东\Desktop\train1.csv'
test_index_path = r'C:\Users\东\Desktop\test1.csv'
train_index=pd.read_csv(train_index_path)['0'].values.astype(np.int)
test_index=pd.read_csv(test_index_path)['0'].values.astype(np.int)
x_img_train=x_img_16[train_index]
x_img_test=x_img_16[test_index]
y_img_train=y_img_16[train_index]
y_img_test=y_img_16[test_index]

100%|███████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 777.26it/s]


In [2]:
###建立模型
import torch.nn.functional as f
from skimage.transform import resize

In [3]:
def build_softmax_module(input_channels):
    return torch.nn.Sequential(
                                torch.nn.Conv2d(in_channels=input_channels,out_channels =1,kernel_size=1),
                                torch.nn.ReLU(),
                                torch.nn.Softmax(dim=2)
                                )

def build_classifier_confidence(input_channels):

    return torch.nn.Sequential(
                                torch.nn.Linear(input_channels,256),
                                torch.nn.ReLU(),
                                torch.nn.Linear(256,1),
                                torch.nn.Tanh()
                                )


class AttentionBlock(torch.nn.Module):
    def __init__(self, input_channels, input_dimension, num_classes):
        super(AttentionBlock, self).__init__()
        self._softmax_block_1 = build_softmax_module(input_channels)
        self._confidence_net = torch.nn.Sequential(
            torch.nn.Linear(input_dimension, 1),
            torch.nn.Tanh()
        )
        self._attention_net = torch.nn.Sequential(
            torch.nn.Linear(input_dimension, num_classes)
        )

        self._attention_heatmaps = [[] for _ in range(num_classes)] #返回num_calsses个空列表

    def forward(self,z:torch.Tensor,y:torch.Tensor,infer:bool):
        heatmap = self._softmax_block_1(z)

        if infer:
            for i, class_ in enumerate(y):
                self._attention_heatmaps[torch.argmax(class_)].append(heatmap[i])
        cross_product = torch.einsum("ijkm,ilkm->ijlkm", (heatmap.clone(), z.clone())) \
            .reshape(heatmap.shape[0], -1,heatmap.shape[2], heatmap.shape[3])
        cross_product = f.avg_pool2d(cross_product.permute(0,3,2,1),(cross_product.shape[2],cross_product.shape[1]))
        cross_product = cross_product.squeeze()
        return self._attention_net(cross_product) * self._confidence_net(cross_product)

from torch.nn import functional as F
class RestNetBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(RestNetBasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        output = self.conv1(x)
        output = F.relu(self.bn1(output))
        output = self.conv2(output)
        output = self.bn2(output)
        return F.relu(x + output)


class RestNetDownBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(RestNetDownBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride[0], padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride[1], padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.extra = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride[0], padding=0),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        extra_x = self.extra(x)
        output = self.conv1(x)
        out = F.relu(self.bn1(output))

        out = self.conv2(out)
        out = self.bn2(out)
        return F.relu(extra_x + out)

class model4(nn.Module):
    def __init__(self, num_of_classes: int, input_dimension: int, uses_attention: bool = True):
        super(model4, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = nn.Sequential(RestNetBasicBlock(64, 64, 1),
                                    RestNetBasicBlock(64, 64, 1))

        self.layer2 = nn.Sequential(RestNetDownBlock(64, 128, [2, 1]),
                                    RestNetBasicBlock(128, 128, 1))

        self.layer3 = nn.Sequential(RestNetDownBlock(128, 256, [2, 1]),
                                    RestNetBasicBlock(256, 256, 1))

        self.layer4 = nn.Sequential(RestNetDownBlock(256, 512, [2, 1]),
                                    RestNetBasicBlock(512, 512, 1))

        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))

        self.fc = nn.Linear(512, num_of_classes)
        if uses_attention:
            print("Model with 2 attention modules.")

            ### build_softmax_module:       conv1d --> ReLU -->Softmax

            ### AttentionBlock: build_softmax_module -->
            self._attention_block_1 = AttentionBlock(64, int(input_dimension / 2), num_of_classes)
            self._attention_block_2 = AttentionBlock(64, int(input_dimension / 2), num_of_classes)
            self._attention_block_3 = AttentionBlock(128, int(input_dimension / 4), num_of_classes)
            self._attention_block_4 = AttentionBlock(256, int(input_dimension / 8), num_of_classes)

            self._classifier_confidence = build_classifier_confidence(512)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.0001)
        self.loss = torch.nn.BCELoss()
        self.uses_attention = uses_attention

    def forward(self, x, y, infer):
        global first_module_prediction, second_module_prediction, \
            third_module_prediction, fourth_module_prediction

        z = self.conv1(x)
        if self.uses_attention:
            first_module_prediction = self._attention_block_1(z, y, infer)
        z = self.layer1(z)
        if self.uses_attention:
            second_module_prediction = self._attention_block_2(z, y, infer)
        z = self.layer2(z)
#         print(z.shape)
        if self.uses_attention:
            third_module_prediction = self._attention_block_3(z, y, infer)
        z = self.layer3(z)
        if self.uses_attention:
            fourth_module_prediction = self._attention_block_4(z, y, infer)
        z = self.layer4(z)
        z = self.avgpool(z)
        z = z.reshape(x.shape[0], -1)
        prediction = self.fc(z)
        # print(prediction.shape)  #(10,5)
        # a = z.view(z.shape[0], -1)
        # b= self._classifier_confidence(z.view(z.shape[0], -1))
        # print(b.shape) #(10,1)
        # print(a.shape) #(10,512)
        if self.uses_attention:
            # print((self._classifier_confidence(z.view(z.shape[0], -1))).shape)
            prediction *= self._classifier_confidence(z.view(z.shape[0], -1))
        if self.uses_attention:
            return f.softmax(prediction + first_module_prediction + second_module_prediction + \
                             third_module_prediction + fourth_module_prediction, dim=1)
        return f.softmax(prediction, dim=1)

model = model4(5,128)


def train_network(x_train: np.ndarray, y_train: np.ndarray, model=model):
    training_history = []
    loss_history = []
    training_time = []
    for current_epoch in range(epochs):
        torch.enable_grad()
        model.train()
        for param in model.parameters():
            param.requires_grad = True

        print("Epoch {}:".format(current_epoch))
        print("\tTraining phase:")

        training_accuracies = []
        losses = []
        begin = time.time()

        x_train_list = [x_train[i:i + batch_size] for i in range(0, len(x_train), batch_size)]
        y_train_list = [y_train[i:i + batch_size] for i in range(0, len(y_train), batch_size)]

        for x, y in tqdm(zip(x_train_list, y_train_list), total=len(x_train_list)):
            x = torch.from_numpy(x.astype("float32")).type(torch.cuda.FloatTensor)
            x = x.permute(0, 3, 1, 2)
            y = torch.from_numpy(y.astype("float32")).type(torch.cuda.LongTensor)
            y = (torch.nn.functional.one_hot(y, num_classes=5)).type(torch.cuda.FloatTensor)
            model.zero_grad()
            model.optimizer.zero_grad()
            out = model(x, y, infer=True)
            loss = model.loss(out, y)
            losses.append(loss.clone().detach().cpu().numpy())
            accuracy = (torch.argmax(out, dim=1) == torch.argmax(y, dim=1)).sum().type(torch.cuda.DoubleTensor) / \
                       y.shape[0]
            #             accuracy = (torch.argmax(out, dim=1) == y).sum().type(torch.cuda.DoubleTensor) / \
            #                        y.shape[0]
            training_accuracies.append(accuracy.cpu().numpy())
            loss.backward()
            model.optimizer.step()

        training_time.append(time.time() - begin)

        loss_history.append(np.mean(losses))
        training_history.append(np.mean(training_accuracies))

        print("\tLoss: {0} ".format(loss_history[-1]))
        print("\tTraining accuracy: {0} ".format(training_history[-1]))

        if not os.path.exists(output_dir):
            os.mkdir(output_dir)
        pickle.dump(training_history,
                    open(os.path.join(output_dir, '2d' + "_training_history.pkl"), "wb"))

        pickle.dump(loss_history, open(os.path.join(output_dir, '2d' + "_loss_history.pkl"), "wb"))
        pickle.dump(training_time,
                    open(os.path.join(output_dir, '2d' + "_time_training.pkl"), "wb"))

        if current_epoch == 0 or (training_history[-1] > max(training_history[:-1])):
            print("\tSaving model...")
            torch.save(model, os.path.join(output_dir, '2d' + "_model.pt"))
        if current_epoch > patience:
            if max(training_history[:-patience]) > max(training_history[-patience:]):
                print("\tBail...")
                break

Model with 2 attention modules.


In [4]:
def train(model,x_train,y_train):
    """
    Method for running the experiments.

    :param args: Parsed arguments.
    :param selected_bands: Bands selected by the outlier detection algorithm.
    :return: None.
    """

    device = torch.device("cpu")
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
        torch.set_default_tensor_type("torch.cuda.FloatTensor")
    os.makedirs(output_dir, exist_ok=True)
    model.to(device)
    train_network(x_train=x_train, y_train=y_train, model=model)
# train(model,x_img_train,y_img_train)

In [5]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt


def infer_network(x_test: np.ndarray, y_test: np.ndarray, model):
    """
    Conduct inference on the trained model.

    :param x_test: Samples for testing.
    :param y_test: Labels for testing.
    :param args: Parsed arguments.
    :param input_size: Size of the initial band spectrum.
    :return: None.
    """
    model = torch.load(os.path.join(output_dir, '2d' + "_model.pt"))
    testing_accuracies = []
    pres = []
    labels = []

    torch.no_grad()
    model.eval()
    for param in model.parameters():
        param.requires_grad = False

    print("\tTesting:")

    begin = time.time()
    x_test_list = [x_test[i:i + batch_size] for i in range(0, len(x_test), batch_size)]
    y_test_list = [y_test[i:i + batch_size] for i in range(0, len(y_test), batch_size)]

    for x, y in tqdm(zip(x_test_list, y_test_list), total=len(x_test_list)):
        x = torch.from_numpy(x.astype("float32")).type(torch.cuda.FloatTensor)
        x = x.permute(0, 3, 1, 2)
        y = torch.from_numpy(y.astype("float32")).type(torch.cuda.LongTensor)
        y = (torch.nn.functional.one_hot(y, num_classes=5)).type(torch.cuda.FloatTensor)
        model.zero_grad()
        accuracy = (torch.argmax(model(x, y, infer=True), dim=1) == torch.argmax(y, dim=1)).sum().type(
            torch.cuda.DoubleTensor) / y.shape[0]
        #         accuracy = (torch.argmax(model(x), dim=1) == torch.argmax(y, dim=1)).sum().type(torch.cuda.DoubleTensor) / y.shape[0]
        testing_accuracies.append(accuracy.cpu().numpy())
        pred1 = torch.argmax(model(x, y, infer=True), dim=1)
        pres = np.append(pres, pred1.cpu().numpy())
        y_t = torch.argmax(y, dim=1)
        labels = np.append(labels, y_t.cpu().numpy())

    testing_time = time.time() - begin
    print("tesing_time:", testing_time)
    testing_accuracy = [np.mean(testing_accuracies)]
    print("\tTesting accuracy: {} ".format(testing_accuracy[0]))
#     if model.uses_attention:
#         heatmaps_per_class = model.get_heatmaps()
#         pickle.dump(heatmaps_per_class,
#                     open(os.path.join(output_dir, '2d' + "_attention_bands.pkl"), "wb"))

#     pickle.dump(testing_accuracy,
#                 open(os.path.join(output_dir, '2d' + "_testing_accuracy.pkl"), "wb"))
#     pickle.dump(testing_time, open(os.path.join(output_dir, '2d' + "_time_testing.pkl"), "wb"))
    cm = confusion_matrix(labels, pres)
    Emotion_kinds = 5  # 这个数值是具体的分类数，大家可以自行修改

    Pre = precision_score(labels, pres, average=None)
    print('Pres:', np.mean(Pre))
    Rec = recall_score(labels, pres, average=None)
    print('Rec:', np.mean(Rec))
    F1 = f1_score(labels, pres, average=None)
    print(np.mean(F1))

    # 显示数据
#     plt.imshow(cm, cmap=plt.cm.Blues)
#     labels = ['N', 'E', 'B', 'L', 'M']  # 每种类别的标签
#     # 在图中标注数量/概率信息
#     thresh = cm.max() / 2  # 数值颜色阈值，如果数值超过这个，就颜色加深。
#     for x in range(Emotion_kinds):
#         for y in range(Emotion_kinds):
#             # 注意这里的matrix[y, x]不是matrix[x, y]
#             info = int(cm[y, x])
#             plt.text(x, y, info,
#                      verticalalignment='center',
#                      horizontalalignment='center',
#                      color="white" if info > thresh else "black")

#     plt.tight_layout()  # 保证图不重叠
#     plt.yticks(range(Emotion_kinds), labels)
#     plt.xticks(range(Emotion_kinds), labels, rotation=45)  # X轴字体倾斜45°
#     plt.show()
#     plt.close()

In [6]:
output_dir = r'E:\2D_A_Result'
batch_size=2
start = time.time()
infer_network(x_img_test,y_img_test,model)
end = time.time()
print(end - start)

  0%|                                                                                          | 0/250 [00:00<?, ?it/s]

	Testing:


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [00:19<00:00, 12.97it/s]


tesing_time: 19.2760591506958
	Testing accuracy: 0.916 
Pres: 0.9154567773723992
Rec: 0.916
0.9151885618030677
41.56687140464783
